*Практика 08.12*

**Выполнить задания:**
1.	Нормализовать базу данных относительно актеров. Выделить таблицу с актерами, а также таблицу многие-ко-многим `актер-фильм`. (Т.е. будут 3 таблицы: актеры, фильмы, актер-фильм. Сохранить новую бд как ‘task1.sqlite’.
2.	Исходя из этой таблицы, средствами SQL, вычислить наболее часто работающую друг с другом пару актеров.


In [ ]:
import pandas as pd
import sqlite3

content = sqlite3.connect('/content/netflix.sqlite')

cur = content.cursor()

In [ ]:
film = pd.read_sql("SELECT * FROM netflix_titles", content)
film.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


### Таблица **Актеры**

In [164]:
def divide_actor(string):
  return string.split(", ")

team = [divide_actor(string) for string in film['cast'].unique()]
replay_team = []
for object in team:
  replay_team += [str(actor) for actor in object]
team = list(set(replay_team))[1:]
team.sort()

In [165]:
actor_id = [int(i) for i in range(len(team))]

actor = pd.DataFrame({
    "actor_id": actor_id,
    "actor_name": team})

In [ ]:
#создаем таблицу actors в БД

with content:
  cur.execute("""
  CREATE TABLE actors (
    actor_id INT,
    actor_name VARCHAR(50),
    PRIMARY KEY(actor_id)
    );
  """)
  content.commit()

In [ ]:
#записываем данные DataFrame

actor.to_sql("actors",
             con=content,
             if_exists="append",
             index=False)
content.commit()

In [ ]:
cur.execute("SELECT * FROM actors").fetchmany()

### Таблица **Фильм** по сути и не нужна, так как netflix.sqlite и является ей.

### Таблица **Актер-Фильм**

In [ ]:
actor_the_film = []
for actor in team:

  #получаем набор фильмов для каждого актера
  film_set = film[film['cast'].apply(lambda x: actor in x.split(", "))]['show_id']

  #создаем DataFrame для каждого актера
  actor_set = [team.index(actor)] * len(film_set.to_list())
  actor_the_film.append(pd.DataFrame({'actor_id': actor_set, 'show_id': film_set.to_list()}))
actor_the_film

In [ ]:
actors_films = pd.concat(actor_the_film)
actors_films['actor_id'] = actors_films['actor_id'].astype (int)
actors_films['show_id'] = actors_films['show_id'].astype (int)
actors_films

In [ ]:
#cоздаем таблицу actors_films в БД

with content:
  cur.execute("""
  CREATE TABLE actors_films (
    actor_id INT,
    show_id INT,
    PRIMARY KEY(actor_id, show_id),
    FOREIGN KEY(actor_id) REFERENCES actors(actor_id),
    FOREIGN KEY(show_id) REFERENCES netflix_titles(show_id)
    );
  """)
  content.commit()

In [ ]:
#записываем данные DataFrame

actors_films.to_sql("actors_films",
                con=content,
                if_exists="append",
                index=False)
content.commit()

In [ ]:
cur.execute("SELECT * FROM actors_films").fetchmany()

### Часто работающая друг с другом пара актеров

In [157]:
content = sqlite3.connect('/content/task1.sqlite')
cur = content.cursor()

In [161]:
together = cur.execute("""
  SELECT af1.actor_id, af2.actor_id, COUNT(*) as frequency
  FROM actors_films af1
  JOIN actors_films af2
      ON af1.show_id = af2.show_id
      AND af1.actor_id != af2.actor_id
  GROUP BY af1.actor_id, af2.actor_id
  ORDER BY frequency DESC
  LIMIT 1;""").fetchone()[1::]
together

(21274, 15)

In [163]:
together = cur.execute("""
    SELECT actor_name FROM actors
    WHERE actor_id = 21274 OR actor_id = 15;
    """).fetchmany(2)
together

[('AJ Bowen',), ('Robb Wells',)]